---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
housing = pd.read_csv('City_Zhvi_AllHomes.csv')
housing.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [4]:
university_towns = pd.read_table('university_towns.txt', header=None)

In [5]:
towns = university_towns.copy()
towns.rename(columns={0:'State'}, inplace=True)
towns['RegionName'] = towns['State']
# Everything that isn't a state, put to NaN in order to use ffill later
towns['State'][~towns['State'].str.contains(r"\[edit\]", regex=True)] = np.nan

# Remove everything in brackets, parantheses and white spaces
towns.replace(r'\[.*\]', '', regex=True, inplace=True)
towns.replace(r'\(.*', '', regex=True, inplace=True)
towns.replace(r'^[ \t]|[ \t]+$', '', regex=True, inplace=True)

# Fill State column and remove rows where State and RegionName are the same 
towns['State'] = towns['State'].ffill()
towns = towns[towns['State'] != towns['RegionName']]

towns = towns.reset_index(drop=True)
towns = towns.sort_values(['State', 'RegionName'])

In [6]:
gdp = pd.read_excel('gdplev.xls', header=5, usecols=[3, 5], names=['Quarter', 'GDP'])
gdp = gdp[gdp['Quarter'].str.contains(r'^2', na=False)]
gdp = gdp.reset_index(drop=True)

gdp.head()

,Quarter,GDP
0,2000q1,12359.1
1,2000q2,12592.5
2,2000q3,12607.7
3,2000q4,12679.3
4,2001q1,12643.3


In [7]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return towns

In [114]:
# import matplotlib as plt
# %matplotlib inline
# gdp.plot(x='Quarter', y='GDP')

In [8]:
gdp_analysis = gdp.copy()
# True if decline
gdp_analysis['Decline'] = gdp_analysis['GDP'] - gdp_analysis['GDP'].shift() < 0

In [9]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    # Index for which we have two declines in a row
    recession_start_index = (gdp_analysis['Decline'] + gdp_analysis['Decline'].shift(-1)).argmax()
    return gdp_analysis['Quarter'][recession_start_index]

recession_start = get_recession_start()

In [10]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    recession_start_index = gdp_analysis[gdp_analysis['Quarter'] == recession_start].index[0]
    # Filter from recession start
    gdp_from_recession_start = gdp_analysis.iloc[recession_start_index:]
    # Index for which we have two growth in a row
    recession_end_index = (~gdp_from_recession_start['Decline'] + (~gdp_from_recession_start['Decline']).shift()).argmax()
    return gdp_from_recession_start['Quarter'][recession_end_index]

recession_end = get_recession_end()

In [11]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    recession_start_index = gdp_analysis[gdp_analysis['Quarter'] == recession_start].index[0]
    recession_end_index = gdp_analysis[gdp_analysis['Quarter'] == recession_end].index[0]
    recession_bottom_index = gdp_analysis['GDP'][recession_start_index:(recession_end_index+1)].argmin()
    return gdp_analysis['Quarter'][recession_bottom_index]

recession_bottom = get_recession_bottom()

In [12]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    house = housing.copy()
    house.replace({'State':states}, inplace=True)
    # All years and months from 2000 to 2016
    keys = [str(year) + '-' + str(month).zfill(2) for year in range(2000,2017) for month in range(1,13)]
    # All quarters from 2000 to 2016 times 3 in order to map to years and months above
    values = [str(x) + 'q' + str(quarter) for x in range(2000,2017) for quarter in range(1,5) for _ in range(3)]
    # Map together and use mapping for grouping in order to create new variables as quarters
    mapping = dict(zip(keys, values))
    return house.set_index(['State', 'RegionName']).groupby(mapping, axis=1).agg(lambda x: x.mean(skipna=True, axis=1))

house = convert_housing_data_to_quarters()

In [129]:
house['RecessionDecline'] = house[recession_start] - house[recession_bottom]

university_indices = list(zip(towns['State'], towns['RegionName']))

university_recession = house.loc[university_indices, 'RecessionDecline'].dropna()
non_university_recession = house.drop(university_indices, errors = 'ignore')['RecessionDecline'].dropna()

p = ttest_ind(university_recession, non_university_recession).pvalue

better = "university town" if university_recession.mean() < non_university_recession.mean() else "non-university town"

In [131]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return (p < 0.01, p, better)

(True, 0.0043252148535112009, 'university town')